In [1]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
app =  dash.Dash()
server =  app.server


In [ ]:
# Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# App
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# layout
app.layout = html.Div([
    
    #header
    
    html.Div([
        html.Div([
            html.Img(src="https://dcassetcdn.com/design_img/3666062/781819/781819_21156990_3666062_e336b60a_image.png",className='four columns')
            
        ], className="four columns"),
        

        html.Div([
            html.H3('Latest Covid 19 Update', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('#StayAtHome', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'right',
                            }),
            
        ], className="four columns")
    ], className="row"),
    
        
    #NavBar
       
     html.Div([
        html.Div([
            html.H3('nav-bar'),
           
        ], className="four columns"),
        html.Div([
            html.H3('nav-bar'),
           
        ], className="four columns"),

        html.Div([
            html.H3('nav-bar'),
            
        ], className="four columns"),
    ], className="row"),
    
      #Band
     html.Div([
       
            html.H3('band')
         
    ], className="row"),
    
    #VISUALISATIONS
     html.Div([
        html.Div([
            html.H3('visualisation'),
            html.Div([
       
                html.H3('Total confirmed cases')
         
            ], className="row"),
            html.Div([
       
            html.H3('Cases by country')
         
    ], className="row"),
            
        ], className="two columns"),
         html.Div([
            html.H3('visualisation'),
            html.Div([
       
            html.H3('World Map')
         
    ], className="row"),
            
        ], className="six columns"),

        html.Div([
            html.H3('visualisation'),
            
            
            html.Div([
                html.Div([
                    html.H3('Total Recoveries'),
                    
                    ], className="six columns"),

                html.Div([
                    html.H3('Total Deaths'),

                    ], className="six columns"),
    ], className="row")
            
        ], className="four columns"),
    ], className="row"),
    
    #fOOTER
     html.Div([
        html.Div([
            html.H3('Footer'),
            
        ], className="six columns"),

        html.Div([
            html.H3('Footer'),
            
        ], className="six columns"),
    ], className="row"),
    
])

if __name__ == '__main__':
    app.run_server()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Apr/2020 14:19:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 14:19:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 14:19:31] "GET /_dash-dependencies HTTP/1.1" 200 -


### Dashboard pages template

In [ ]:
# # Style sheet
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # App
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # layout
# app.layout = html.Div([
    
#     #header
    
#     html.Div([
#         html.Div([
#             html.H3('Header'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('header'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('header'),
            
#         ], className="four columns")
#     ], className="row"),
    
        
#     #NavBar
       
#      html.Div([
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),

#         html.Div([
#             html.H3('nav-bar'),
            
#         ], className="four columns"),
#     ], className="row"),
    
#       #Band
#      html.Div([
       
#             html.H3('band')
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),
    
# ])

# if __name__ == '__main__':
#     app.run_server()
